In [1]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='iso-8859-1')
        return data['data'],data['labels']

#tensorflow.Dataset
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self,batch_size):
        """:return batch_szie examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i ) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,True)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [11]:
def residual_block(X, output_channel):
    """residual connection implementation(残差连接块)"""
    input_channel = X.get_shape().as_list()[-1]
    if input_channel * 2 == output_channel:
        increase_dim = True
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim = False
        strides = (1,1)
    else:
        raise Exception("input channel can't match output channel")

    conv1 = tf.layers.conv2d(X,
                             output_channel,
                             (3,3),
                             strides = strides,
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv1')

    conv2 = tf.layers.conv2d(conv1,
                             output_channel,
                             (3,3),
                             strides = (1,1),
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv2')

    if increase_dim:
        # [None, image_width, image_height, channel] -> [,,,channel * 2]
        pooled_X = tf.layers.average_pooling2d(X,
                                     (2,2),
                                     (2,2),
                                     padding = 'valid')
        padded_X = tf.pad(pooled_X,
                          [[0,0],
                          [0,0],
                          [0,0],
                          [input_channel // 2, input_channel // 2]])
    else:
        padded_X = X
    output_X = conv2 + padded_X
    return output_X

def res_net(X,
            num_residual_blocks,
            num_filter_base,
            class_num):
    """residual network implementation"""
    """
    :param
    -X:
    -num_residual_blocks: 残差连接块数目:eg:[3,4,6,3]
    -num_subsampling:降采样数目,与残差连接块数目相等,可以从上个参数获得
    -num_filter_base:原始输出通道数
    -class_num:多分类的类别数
    """
    num_subsampling = len(num_residual_blocks)
    layers = []
    # X: [none,width,height,channel] -> [width,height,channel]
    input_size = X.get_shape().as_list()[1:]
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(X,
                                 num_filter_base,
                                 (3,3),
                                 strides = (1,1),
                                 padding='same',
                                 activation = tf.nn.relu,
                                 name = 'conv0')
        layers.append(conv0)
    # num_subsampling = 4, sample_id = [0,1,2,3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],
                    num_filter_base * (2 ** sample_id))
                layers.append(conv)

    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]

    with tf.variable_scope('fc'):
        # layer[-1].shape: [None, width, height, channel]
        # kernal_size: image_width, image_height 经过global_pooling后就变为一个值,就是他的均值
        global_pool = tf.reduce_mean(layers[-1], [1,2])
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
        return layers[-1]


X = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])
# [None], eg: [0,5,6,3]
X_image = tf.reshape(X, [-1,3,32,32])
# 32 * 32
X_image = tf.transpose(X_image, perm = [0,2,3,1])

# conv1: 神经元图, feature_map, 输出图像
with tf.variable_scope("encoder",reuse=tf.AUTO_REUSE) as scope:


    y_ = res_net(X_image, [2,3,2], 32, 10)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
    # y_ -> softmax
    # y -> one_hot
    # loss = ylogy_

    # indices
    predict = tf.argmax(y_,1)
    # [1,0,1,1,1,0,0,0]
    correct_prediction = tf.equal(predict,y)
    accuary = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

    with tf.name_scope('train_op'):
        # 反向传播
        train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


[]


In [8]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,accu_val ,_ =sess.run(
            [loss,accuary,train_op],
            feed_dict={
                X:batch_data,
                y:batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step :%d, loss: %4.5f, acc: %4.5f'\
                  %(i+1,loss_val,accu_val))

        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data ,test_batch_labels\
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuary],
                    feed_dict = {
                        X: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.nanmean(all_test_acc_val)
            print('[Test ] Step :%d, acc: %4.5f'\
                  %(i+1,test_acc))

[Train] Step :100, loss: 2.06986, acc: 0.20000
[Train] Step :200, loss: 1.89311, acc: 0.30000
[Train] Step :300, loss: 1.98209, acc: 0.30000
[Train] Step :400, loss: 1.65589, acc: 0.35000
[Train] Step :500, loss: 1.72418, acc: 0.35000
[Train] Step :600, loss: 1.71867, acc: 0.35000
[Train] Step :700, loss: 1.80920, acc: 0.45000
[Train] Step :800, loss: 1.56955, acc: 0.30000
[Train] Step :900, loss: 1.52584, acc: 0.40000
[Train] Step :1000, loss: 1.63714, acc: 0.30000
(10000, 3072)
(10000,)
[Test ] Step :1000, acc: 0.42000
